# Using an auto encoder on MNIST handwritten digits.¶

In [1]:
import org.apache.log4j.{Level, Logger}
import org.apache.spark.SparkContext

import com.intel.analytics.bigdl.utils._
import com.intel.analytics.bigdl.utils.{Engine, LoggerFilter, T, Table}
import com.intel.analytics.bigdl.dataset.DataSet
import com.intel.analytics.bigdl.dataset.image.{BytesToGreyImg, GreyImgNormalizer, GreyImgToBatch, GreyImgToSample}
import com.intel.analytics.bigdl.models.lenet.Utils._
import com.intel.analytics.bigdl.models.autoencoder._
import com.intel.analytics.bigdl.nn._
import com.intel.analytics.bigdl.optim._
import com.intel.analytics.bigdl.optim.{Adam, Top1Accuracy, Trigger}
import com.intel.analytics.bigdl.visualization.{TrainSummary, ValidationSummary}
import com.intel.analytics.bigdl.tensor.Tensor
import com.intel.analytics.bigdl.numeric.NumericFloat

Engine.init

## 1. Load MNIST dataset

In [2]:
import java.nio.ByteBuffer
import java.nio.file.{Files, Path, Paths}

import com.intel.analytics.bigdl.dataset.ByteRecord
import com.intel.analytics.bigdl.utils.File
import scopt.OptionParser

def load(featureFile: String, labelFile: String): Array[ByteRecord] = {
    val featureBuffer = ByteBuffer.wrap(Files.readAllBytes(Paths.get(featureFile)))
    val labelBuffer = ByteBuffer.wrap(Files.readAllBytes(Paths.get(labelFile)))
    
    val labelMagicNumber = labelBuffer.getInt()
    require(labelMagicNumber == 2049)
    val featureMagicNumber = featureBuffer.getInt()
    require(featureMagicNumber == 2051)

    val labelCount = labelBuffer.getInt()
    val featureCount = featureBuffer.getInt()
    require(labelCount == featureCount)

    val rowNum = featureBuffer.getInt()
    val colNum = featureBuffer.getInt()

    val result = new Array[ByteRecord](featureCount)
    var i = 0
    while (i < featureCount) {
      val img = new Array[Byte]((rowNum * colNum))
      var y = 0
      while (y < rowNum) {
        var x = 0
        while (x < colNum) {
          img(x + y * colNum) = featureBuffer.get()
          x += 1
        }
        y += 1
      }
      result(i) = ByteRecord(img, labelBuffer.get().toFloat + 1.0f)
      i += 1
    }

    result
}

Then we need to set paths of data. Please edit paths if they are changed.

In [3]:
val trainData = "../../datasets/mnist/train-images-idx3-ubyte"
val trainLabel = "../../datasets/mnist/train-labels-idx1-ubyte"
val validationData = "../../datasets/mnist/t10k-images-idx3-ubyte"
val validationLabel = "../../datasets/mnist/t10k-labels-idx1-ubyte"

## 2. Model Setup

In [4]:
//Parameters
val batchSize = 128
val maxEpochs = 2
val displayStep = 1
val examples2Show = 10

//Network Parameters
val nInput = 784 //MNIST data input (img shape: 28*28)
val nHidden = 32 // hidden layer num of features

Then the data set should be created and the model needs to be established.

In [5]:
val trainMean = 0.13066047740239436
val trainStd = 0.30810779333114624
val trainSet = 
    DataSet.array(load(trainData, trainLabel), sc) -> BytesToGreyImg(28, 28) -> GreyImgNormalizer(trainMean, trainStd) -> GreyImgToBatch(batchSize) -> toAutoencoderBatch()
val validationSet = 
    DataSet.array(load(validationData, validationLabel), sc) -> BytesToGreyImg(28, 28) -> GreyImgNormalizer(testMean, testStd) -> GreyImgToBatch(batchSize) -> toAutoencoderBatch()

In [6]:
var model = Sequential()
model.add(Reshape(Array(28*28)))
// encoder
model.add(Linear(nInput, nHidden))
model.add(ReLU[Float]())
// decoder
model.add(Linear(nHidden, nInput))
model.add(Sigmoid[Float]())

Sequential[d3fa8b9b]{
  [input -> (1) -> (2) -> (3) -> (4) -> (5) -> output]
  (1): Reshape[a4e0b228](784)
  (2): Linear[d8a294c5](784 -> 32)
  (3): ReLU[678cd7a1](0.0, 0.0)
  (4): Linear[f62a17fc](32 -> 784)
  (5): Sigmoid[66c5634b]
}

## 3. Optimizer Setup

In [7]:
val optimizer = Optimizer(model = model, dataset = trainSet, criterion = new MSECriterion[Float]())
optimizer.setOptimMethod(new Adam())
optimizer.setEndWhen(Trigger.maxEpoch(maxEpochs))

com.intel.analytics.bigdl.optim.DistriOptimizer@2d7621e1

The following is to create training and validation summary.

In [8]:
import java.text.SimpleDateFormat
import java.util.Calendar
val today = Calendar.getInstance
val formatDate = new SimpleDateFormat("yyyyMMdd-hhmmss")
val name = "autoencoder-" + formatDate.format(today.getTime()).toString()
val trainSummary = TrainSummary(logDir="/tmp/bigdl_summaries", appName=name)
trainSummary.setSummaryTrigger("Parameters", Trigger.severalIteration(50))
val valSummary = ValidationSummary(logDir="/tmp/bigdl_summaries", appName=name)
optimizer.setTrainSummary(trainSummary)
optimizer.setValidationSummary(valSummary)
printf("saving logs to %s", name)

saving logs to autoencoder-20170925-041354

In [9]:
// Boot training process
val trainedModel = optimizer.optimize()
print("Optimization Done.")

Optimization Done.

In [10]:
import vegas._
import vegas.render.HTMLRenderer._
import vegas.DSL._

In [11]:
val loss = trainSummary.readScalar("Loss")
val lossXY = loss.map(_ ._1).zip(loss.map(_ ._2)).toSeq
Vegas(description = "The Loss curve.", width = 700.0, height = 300.0).
  withXY(lossXY).
  encodeX("x", Quantitative, bin = Bin(maxbins = 500.0), title = "Iteration").
  encodeY("y", Quantitative, title = "Loss").
  mark(Line).
  show

<iframe id="frame-vegas-4f71130e-504a-4f38-89ef-373affa022cb" sandbox="allow-scripts allow-same-origin" style="border: none; width: 100%" srcdoc="<html>
 <head>
 <script src="https://cdn.jsdelivr.net/webjars/org.webjars.bower/d3/3.5.17/d3.min.js" charset="utf-8"></script>
<script src="https://cdn.jsdelivr.net/webjars/org.webjars.bower/vega/2.6.3/vega.min.js" charset="utf-8"></script>
<script src="https://cdn.jsdelivr.net/webjars/org.webjars.bower/vega-lite/1.2.0/vega-lite.min.js" charset="utf-8"></script>
<script src="https://vega.github.io/vega-editor/vendor/vega-embed.js" charset="utf-8"></script>
 </head>
 <body>
 <div id='vegas-4f71130e-504a-4f38-89ef-373affa022cb'></div>
 <script>
 var embedSpec = {
 mode: "vega-lite",
 spec: {
 "width" : 700.0,
 "height" : 300.0,
 "mark" : "line",
 "encoding" : {
 "x" : {
 "field" : "x",
 "type" : "quantitative",
 "bin" : {
 "maxbins" : 500.0
 },
 "title" : "Iteration"
 },
 "y" : {
 "field" : "y",
 "type" : "quantitative",
 "title" : "Loss"
 }
 },
 "description" : "The Loss curve.",
 "data" : {
 "values" : [
 {
 "x" : 1,
 "y" : 1.249737024307251
 },
 {
 "x" : 2,
 "y" : 1.2355479001998901
 },
 {
 "x" : 3,
 "y" : 1.25474214553833
 },
 {
 "x" : 4,
 "y" : 1.2396063804626465
 },
 {
 "x" : 5,
 "y" : 1.2326170206069946
 },
 {
 "x" : 6,
 "y" : 1.2523821592330933
 },
 {
 "x" : 7,
 "y" : 1.2038089036941528
 },
 {
 "x" : 8,
 "y" : 1.2174901962280273
 },
 {
 "x" : 9,
 "y" : 1.197322130203247
 },
 {
 "x" : 10,
 "y" : 1.1983468532562256
 },
 {
 "x" : 11,
 "y" : 1.2089793682098389
 },
 {
 "x" : 12,
 "y" : 1.1852822303771973
 },
 {
 "x" : 13,
 "y" : 1.1399346590042114
 },
 {
 "x" : 14,
 "y" : 1.1221776008605957
 },
 {
 "x" : 15,
 "y" : 1.1267306804656982
 },
 {
 "x" : 16,
 "y" : 1.1095027923583984
 },
 {
 "x" : 17,
 "y" : 1.119856834411621
 },
 {
 "x" : 18,
 "y" : 1.0813980102539062
 },
 {
 "x" : 19,
 "y" : 1.0860868692398071
 },
 {
 "x" : 20,
 "y" : 1.0615636110305786
 },
 {
 "x" : 21,
 "y" : 1.0479084253311157
 },
 {
 "x" : 22,
 "y" : 1.055711269378662
 },
 {
 "x" : 23,
 "y" : 1.024783968925476
 },
 {
 "x" : 24,
 "y" : 1.0061086416244507
 },
 {
 "x" : 25,
 "y" : 0.9920810461044312
 },
 {
 "x" : 26,
 "y" : 0.9894428253173828
 },
 {
 "x" : 27,
 "y" : 0.962231457233429
 },
 {
 "x" : 28,
 "y" : 0.9537112712860107
 },
 {
 "x" : 29,
 "y" : 0.9333881735801697
 },
 {
 "x" : 30,
 "y" : 0.951988160610199
 },
 {
 "x" : 31,
 "y" : 0.9227176904678345
 },
 {
 "x" : 32,
 "y" : 0.9279816746711731
 },
 {
 "x" : 33,
 "y" : 0.9122300744056702
 },
 {
 "x" : 34,
 "y" : 0.901889443397522
 },
 {
 "x" : 35,
 "y" : 0.8454015851020813
 },
 {
 "x" : 36,
 "y" : 0.8907661437988281
 },
 {
 "x" : 37,
 "y" : 0.8566211462020874
 },
 {
 "x" : 38,
 "y" : 0.8707225322723389
 },
 {
 "x" : 39,
 "y" : 0.8600653409957886
 },
 {
 "x" : 40,
 "y" : 0.849868893623352
 },
 {
 "x" : 41,
 "y" : 0.8479325175285339
 },
 {
 "x" : 42,
 "y" : 0.8379390835762024
 },
 {
 "x" : 43,
 "y" : 0.8224451541900635
 },
 {
 "x" : 44,
 "y" : 0.8114774823188782
 },
 {
 "x" : 45,
 "y" : 0.8086097240447998
 },
 {
 "x" : 46,
 "y" : 0.7779930830001831
 },
 {
 "x" : 47,
 "y" : 0.8256412148475647
 },
 {
 "x" : 48,
 "y" : 0.8256152272224426
 },
 {
 "x" : 49,
 "y" : 0.8005053400993347
 },
 {
 "x" : 50,
 "y" : 0.7976285219192505
 },
 {
 "x" : 51,
 "y" : 0.7651920318603516
 },
 {
 "x" : 52,
 "y" : 0.8146206736564636
 },
 {
 "x" : 53,
 "y" : 0.7862396836280823
 },
 {
 "x" : 54,
 "y" : 0.772801399230957
 },
 {
 "x" : 55,
 "y" : 0.7962597608566284
 },
 {
 "x" : 56,
 "y" : 0.7908921241760254
 },
 {
 "x" : 57,
 "y" : 0.7664347290992737
 },
 {
 "x" : 58,
 "y" : 0.7474218010902405
 },
 {
 "x" : 59,
 "y" : 0.7733684182167053
 },
 {
 "x" : 60,
 "y" : 0.7706124782562256
 },
 {
 "x" : 61,
 "y" : 0.7437905073165894
 },
 {
 "x" : 62,
 "y" : 0.7868576049804688
 },
 {
 "x" : 63,
 "y" : 0.7459282875061035
 },
 {
 "x" : 64,
 "y" : 0.7359863519668579
 },
 {
 "x" : 65,
 "y" : 0.7645328640937805
 },
 {
 "x" : 66,
 "y" : 0.760829746723175
 },
 {
 "x" : 67,
 "y" : 0.7472641468048096
 },
 {
 "x" : 

Finally, the Spark should be stopped.

In [12]:
sc.stop()